# 1. Instanciate URIs for terms (legislatura) 

# 3. Allocate Posts and Memberships

 Solves URI to Post as interval scheduling problem.

In [59]:
import json 
import pandas as pd
import sys
from collections import deque

start_date = '2015-02-01'

In [60]:
with open('datasets/posts-55.csv', mode='r') as f:
    postsstr = f.read()
f.close()
posts = postsstr.split('\n')[1:]
postsdeque_dict = {start_date:deque(posts, maxlen=len(posts))} 
print(len(posts))

315


In [61]:
with open('datasets/membership_with_role_deputado-55.json', mode='r') as f:
    membership_with_role_deputado = json.load(f)
f.close()
df = pd.DataFrame.from_dict(membership_with_role_deputado)
df.head()

,cam:dataFim,cam:dataInicio,cam:descricaoCausaFimExercicio,cam:idCadastroParlamentarAnterior,cam:idCausaFimExercicio,cam:siglaUFRepresentacao,cam:situacaoExercicio,org:hasMember,org:role,slp:resource_uri
0,2017-11-22,2015-02-01,Ministro de Estado,None,11,GO,Titular,1281c859-0110-4827-ad8e-e3311a35fb36,b27beba7-ca02-4041-a9e0-1793bcd141fe,d04a7fcd-9047-45b9-a2d5-edfc6fa2d544
1,None,2015-02-01,None,None,None,RJ,Titular,7fec3c12-9ff7-446b-909c-956eedf58896,b27beba7-ca02-4041-a9e0-1793bcd141fe,fd6d3547-a8b7-4081-9dd3-2a67abc89237
2,None,2015-02-01,None,None,None,PB,Titular,b1e02350-c537-4998-8d9d-9f6ed74a899e,b27beba7-ca02-4041-a9e0-1793bcd141fe,38e300fb-a77d-46a2-b88d-6e171ce10468
3,2017-01-01,2015-02-01,Renúncia,None,16,SP,Titular,353a7f98-7b7b-43fa-9c2d-665713f1d0f5,b27beba7-ca02-4041-a9e0-1793bcd141fe,5597f222-7c84-4d3d-ae2f-f2675ce58689
4,None,2015-02-01,None,None,None,PA,Titular,a589b3f1-96d6-4e6b-8da0-25cbcb9f6de6,b27beba7-ca02-4041-a9e0-1793bcd141fe,7efcd22f-c948-4ff2-b774-b1c8419d565d


In [62]:
def sortfinish(x):
    if not x['cam:dataFim']:
        return '1800-01-01'
    return x['cam:dataFim']

def sortstart(x):
    return x['cam:dataInicio']

def pop(process_date):
    key = min([k for k in postsdeque_dict 
               if ((k >= process_date) and (len(postsdeque_dict[k]) > 0))])
    return postsdeque_dict[key] 

def push(process_date, this_post):
    if process_date not in postsdeque_dict:
        postsdeque_dict[process_date] = deque([this_post] , maxlen=315)
    else:
        postsdeque_dict[process_date] = postsdeque_dict[process_date].push(this_post) 
    return postsdeque_dict[process_date]


def peeklen(process_date):
    keys = min([k for k in postsdeque_dict 
               if ((k >= process_date) and (len(postsdeque_dict[k]) > 0))])
    return sum([len(postsdeque_dict[k]) for k in keys])

In [63]:
membership_with_role_deputado2 = sorted(
    sorted( membership_with_role_deputado,
            key=sortstart),
    key=sortfinish
)
df = pd.DataFrame.from_dict(membership_with_role_deputado2)
df.head()

,cam:dataFim,cam:dataInicio,cam:descricaoCausaFimExercicio,cam:idCadastroParlamentarAnterior,cam:idCausaFimExercicio,cam:siglaUFRepresentacao,cam:situacaoExercicio,org:hasMember,org:role,slp:resource_uri
0,None,2015-02-01,None,None,None,RJ,Titular,7fec3c12-9ff7-446b-909c-956eedf58896,b27beba7-ca02-4041-a9e0-1793bcd141fe,fd6d3547-a8b7-4081-9dd3-2a67abc89237
1,None,2015-02-01,None,None,None,PB,Titular,b1e02350-c537-4998-8d9d-9f6ed74a899e,b27beba7-ca02-4041-a9e0-1793bcd141fe,38e300fb-a77d-46a2-b88d-6e171ce10468
2,None,2015-02-01,None,None,None,PA,Titular,a589b3f1-96d6-4e6b-8da0-25cbcb9f6de6,b27beba7-ca02-4041-a9e0-1793bcd141fe,7efcd22f-c948-4ff2-b774-b1c8419d565d
3,None,2015-02-01,None,None,None,AC,Titular,db882a61-939d-4a79-954e-9752e1eb808a,b27beba7-ca02-4041-a9e0-1793bcd141fe,73b98b4a-5e3a-44cc-b46a-20047b7c92b1
4,None,2015-02-01,None,None,None,RS,Titular,58832b7d-e94e-40a8-a840-500cd103ba58,b27beba7-ca02-4041-a9e0-1793bcd141fe,88c8c684-df6e-4d56-8b97-f989c3c53344


In [64]:
# num_posts = 315
with open('role_deputado_with_posts-55.csv', mode='w') as f:
    f.write('org:role;org:post\n')
    
    for i, d in enumerate(membership_with_role_deputado2):
        processing_date = d['cam:dataInicio']
        finish_date = d['cam:dataFim']
        
        # pops from processing_date
        this_post = pop(processing_date)
        num_posts = peeklen(processing_date)
        
        sys.stdout.write('{:}:processing_date:{:}\tnum_posts:{:}\r'.format(i + 1, processing_date, num_posts))
        # pushes on finish_date
        push(processing_date, this_post)
                
        buff = '{:};{:}\n'.format(d['org:role'] , this_post)
        f.write('{:};{:}\n')


NameError: name 'process_date' is not defined